In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from model import SimSiam
from utils import create_linear_classifier, load_pretrained_encoder


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimSiam(backbone='resnet18').to(device)

In [ ]:
# --- Load pretrained encoder ---
encoder = load_pretrained_encoder('./checkpoints_new_backbone/simsiam_encoder.pth', backbone='resnet18')
model = create_linear_classifier(encoder, num_classes=100, freeze_encoder=True).to('cuda')

# --- CIFAR-10 Dataloaders ---

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

train_set = datasets.CIFAR10(root='../data', train=True, transform=transform, download=False)
test_set = datasets.CIFAR10(root='../data', train=False, transform=transform, download=False)

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4)

# --- Training setup ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model[-1].parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [4]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

writer = SummaryWriter(log_dir='../runs/linear_eval_cifar_10_no_transform')

# --- Train only linear head ---
for epoch in range(200):
    model.train()
    correct, total, total_loss = 0, 0, 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1:03d}", leave=False)
    for x, y in pbar:
        x, y = x.to('cuda'), y.to('cuda')

        logits = model(x)
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

        pbar.set_postfix({
            'Loss': f"{loss.item():.4f}",
            'Acc': f"{100 * correct / total:.2f}%"
        })

    acc = 100 * correct / total
    writer.add_scalar('Train/Loss', total_loss / len(train_loader), epoch + 1)
    writer.add_scalar('Train/Accuracy', acc, epoch + 1)

    print(f"Epoch {epoch+1:3d} | Loss: {total_loss:.4f} | Train Acc: {acc:.2f}%")

    # --- Evaluation on Test Set every 100 epochs ---
    if (epoch + 1) % 100 == 0:
        model.eval()
        test_correct, test_total = 0, 0

        pbar = tqdm(test_loader, desc=f"Evaluating @ Epoch {epoch+1}", leave=False)
        with torch.no_grad():
            for x, y in pbar:
                x, y = x.to('cuda'), y.to('cuda')
                preds = model(x).argmax(dim=1)
                test_correct += (preds == y).sum().item()
                test_total += y.size(0)

        test_acc = 100 * test_correct / test_total
        writer.add_scalar('Test/Accuracy', test_acc, epoch + 1)
        print(f"Test Accuracy @ Epoch {epoch+1:3d}: {test_acc:.2f}%")

# --- Final Evaluation on Test Set ---
model.eval()
correct, total = 0, 0

pbar = tqdm(test_loader, desc="Final Evaluation", leave=False)
with torch.no_grad():
    for x, y in pbar:
        x, y = x.to('cuda'), y.to('cuda')
        preds = model(x).argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

final_test_acc = 100 * correct / total
writer.add_scalar('Test/Accuracy', final_test_acc, 400)  # Also log final eval
print(f"\nFinal Linear Evaluation Accuracy on CIFAR-10: {final_test_acc:.2f}%")

# --- Save final model weights ---
torch.save(model.state_dict(), 'linear_head_cifar10_final_weights.pth')
print("Final model weights saved as 'linear_head_cifar10_final_weights.pth'")

Epoch   1 | Loss: 756.6411 | Train Acc: 31.72%


Epoch   2 | Loss: 691.2498 | Train Acc: 38.05%


Epoch   3 | Loss: 665.9612 | Train Acc: 40.75%


Epoch   4 | Loss: 648.0638 | Train Acc: 42.59%


Epoch   5 | Loss: 634.5728 | Train Acc: 43.83%


...

Epoch 198 | Loss: 481.2182 | Train Acc: 56.67%


Epoch 199 | Loss: 481.7671 | Train Acc: 56.65%


Epoch 200 | Loss: 480.6632 | Train Acc: 56.62%


Test Accuracy @ Epoch 200: 55.19%



Final Linear Evaluation Accuracy on CIFAR-10: 55.19%
Final model weights saved as 'linear_head_cifar10_final_weights.pth'
